In [18]:
import os
import re
import shutil
import json

import numpy as np
import parmed as pmd

from paprika import align
from paprika import amber
from paprika import analysis
from paprika import io
from paprika import log
from paprika import restraints
from paprika import tleap


In [19]:
log.config_root_logger(verbose=True)

# Specify directory for data

In this case, we just need some initial coordinates. In other cases, we might need `mol2` or `frcmod` files.

In [20]:
k_cl_pdb = os.path.abspath(os.path.join("data", "k-cl.pdb"))

# Setup the calcuation

## Build the vacuum `prmtop` and `inpcrd` files

In [21]:
# Build the model in vacuum

sys = tleap.System()
sys.template_lines = [
    "source leaprc.water.tip3p",
    "loadamberparams frcmod.ionsjc_tip3p",
    f"model = loadpdb {k_cl_pdb}",
]

sys.output_path = "k-cl"
sys.output_prefix = "k-cl"
sys.pbc_type = None
sys.target_waters = None
sys.neutralize = False
sys.build()

2019-03-21 22:32:54,558: Running tleap.build() in k-cl


## Specify the number of windows for the umbrella sampling

These are overkill; I have been testing how much data we need to converge this calculation and how quickly we can run a stripped-down version on Travis.

In [22]:
# Setup the windows

attach_fractions = np.linspace(0, 1.0, 5)
initial_distance = 2.65
pull_distances = np.linspace(0 + initial_distance, 16.0 + initial_distance, 10)

## Add a single distance restraint between K+ and Cl-

In [23]:
# Setup the single distance restraint

restraint = restraints.DAT_restraint()
restraint.continuous_apr = True
restraint.amber_index = True
restraint.topology = k_cl_pdb
restraint.mask1 = "@K+"
restraint.mask2 = "@Cl-"

restraint.attach["target"] = initial_distance
restraint.attach["fraction_list"] = attach_fractions
restraint.attach["fc_final"] = 5.0
restraint.pull["fc"] = restraint.attach["fc_final"]
restraint.pull["target_list"] = pull_distances
restraint.initialize()

2019-03-21 22:32:54,866: Calculating attach targets and force constants...
2019-03-21 22:32:54,868: Attach, Method #3
2019-03-21 22:32:54,869: Calculating pull targets and force constants...
2019-03-21 22:32:54,870: Pull, Method #5
2019-03-21 22:32:54,871: Calculating release targets and force constants...
2019-03-21 22:32:54,872: No restraint info set for the release phase! Skipping...
2019-03-21 22:32:54,873: Number of attach windows = 5
2019-03-21 22:32:54,875: Number of pull windows = 10
2019-03-21 22:32:54,876: This restraint will be skipped in the release phase
2019-03-21 22:32:54,880: Assigning atom indices...
2019-03-21 22:32:54,886: Loaded /Users/dslochower/Documents/projects/paprika-kcl/data/k-cl.pdb...
2019-03-21 22:32:54,886: Loaded /Users/dslochower/Documents/projects/paprika-kcl/data/k-cl.pdb...
2019-03-21 22:32:54,886: Loaded /Users/dslochower/Documents/projects/paprika-kcl/data/k-cl.pdb...
2019-03-21 22:32:54,890: There are 1 atoms in the mask @K+  ...
2019-03-21 22:32:

## Optionally, add a "wall restraint" to define the bound state and speed convergence

This will apply a harmonic potential that keeps the "guest" Cl- within 3.5 Angstroms.

In [24]:
# Add wall restraint during attachment

wall = restraints.DAT_restraint()
wall.auto_apr = False
wall.amber_index = True
wall.topology = k_cl_pdb
wall.mask1 = "@K+"
wall.mask2 = "@Cl-"

wall.attach["fc_initial"] = 1.0
wall.attach["fc_final"] = 1.0

wall.custom_restraint_values["rk2"] = 1.0
wall.custom_restraint_values["rk3"] = 1.0
wall.custom_restraint_values["r1"] = 0.0
wall.custom_restraint_values["r2"] = 3.5
wall.custom_restraint_values["r3"] = 3.5
wall.custom_restraint_values["r4"] = 999

wall.attach["target"] = 3.5
wall.attach["num_windows"] = len(attach_fractions)

wall.initialize()

2019-03-21 22:32:54,917: Calculating attach targets and force constants...
2019-03-21 22:32:54,919: Attach, Method #1
2019-03-21 22:32:54,921: Calculating pull targets and force constants...
2019-03-21 22:32:54,922: No restraint info set for the pull phase! Skipping...
2019-03-21 22:32:54,923: Calculating release targets and force constants...
2019-03-21 22:32:54,925: No restraint info set for the release phase! Skipping...
2019-03-21 22:32:54,926: Number of attach windows = 5
2019-03-21 22:32:54,927: This restraint will be skipped in the pull phase
2019-03-21 22:32:54,930: This restraint will be skipped in the release phase
2019-03-21 22:32:54,932: Assigning atom indices...
2019-03-21 22:32:54,935: Loaded /Users/dslochower/Documents/projects/paprika-kcl/data/k-cl.pdb...
2019-03-21 22:32:54,935: Loaded /Users/dslochower/Documents/projects/paprika-kcl/data/k-cl.pdb...
2019-03-21 22:32:54,935: Loaded /Users/dslochower/Documents/projects/paprika-kcl/data/k-cl.pdb...
2019-03-21 22:32:54,93

## Create the directories for each window and write the AMBER-style restraint input file

This makes it easy to run each window in parallel as a separate simulation.

In [25]:
windows_directory = os.path.join("k-cl", "windows")

In [26]:
# Create the windows
window_list = restraints.create_window_list([restraint])
for window in window_list:
    if os.path.exists(os.path.join(windows_directory, window)):
        continue
    else:
        os.makedirs(os.path.join(windows_directory, window))

2019-03-21 22:32:54,971: All restraints are "continuous_apr" style.
2019-03-21 22:32:54,975: Restraints appear to be consistent
2019-03-21 22:32:54,975: Restraints appear to be consistent
2019-03-21 22:32:54,975: Restraints appear to be consistent


In [27]:
for window in window_list:
    with open(os.path.join(windows_directory, window, "disang.rest"), "a") as file:
        if window[0] == "a":
            for r in [restraint, wall]:
                string = restraints.amber_restraint_line(r, window)
                if string is not None:
                    file.write(string)
        else:
            string = restraints.amber_restraint_line(restraint, window)
            file.write(string)

2019-03-21 22:32:55,001: Overriding rk2 = 1.0
2019-03-21 22:32:55,003: Overriding rk3 = 1.0
2019-03-21 22:32:55,006: Overriding r1 = 0.0
2019-03-21 22:32:55,007: Overriding r2 = 3.5
2019-03-21 22:32:55,009: Overriding r3 = 3.5
2019-03-21 22:32:55,016: Overriding r4 = 999
2019-03-21 22:32:55,021: Overriding rk2 = 1.0
2019-03-21 22:32:55,023: Overriding rk3 = 1.0
2019-03-21 22:32:55,025: Overriding r1 = 0.0
2019-03-21 22:32:55,027: Overriding r2 = 3.5
2019-03-21 22:32:55,028: Overriding r3 = 3.5
2019-03-21 22:32:55,034: Overriding r4 = 999
2019-03-21 22:32:55,037: Overriding rk2 = 1.0
2019-03-21 22:32:55,039: Overriding rk3 = 1.0
2019-03-21 22:32:55,041: Overriding r1 = 0.0
2019-03-21 22:32:55,043: Overriding r2 = 3.5
2019-03-21 22:32:55,044: Overriding r3 = 3.5
2019-03-21 22:32:55,050: Overriding r4 = 999
2019-03-21 22:32:55,051: Overriding rk2 = 1.0
2019-03-21 22:32:55,054: Overriding rk3 = 1.0
2019-03-21 22:32:55,055: Overriding r1 = 0.0
2019-03-21 22:32:55,057: Overriding r2 = 3.5
20

In [28]:
for window in window_list:
    if window[0] == "a":
        structure = pmd.load_file(os.path.join("k-cl", "k-cl.prmtop"), 
                                  os.path.join("k-cl", "k-cl.rst7"), structure=True)
        for atom in structure.atoms:
            if atom.name == "Cl-":
                atom.xz = 2.65
        structure.save(os.path.join(windows_directory, window, "k-cl.prmtop"), overwrite=True)
        structure.save(os.path.join(windows_directory, window, "k-cl.rst7"), overwrite=True)

    elif window[0] == "p":
        structure = pmd.load_file(os.path.join("k-cl", "k-cl.prmtop"), 
                                  os.path.join("k-cl", "k-cl.rst7"), structure=True)
        target_difference = (
            restraint.phase["pull"]["targets"][int(window[1:])]
            - restraint.phase["pull"]["targets"][0]
        )
        print(
            f"In window {window} we will translate the guest {target_difference:0.1f} Angstroms."
        )
        for atom in structure.atoms:
            if atom.name == "Cl-":
                atom.xz += target_difference
        structure.save(os.path.join(windows_directory, window, "k-cl.prmtop"), overwrite=True)
        structure.save(os.path.join(windows_directory, window, "k-cl.rst7"), overwrite=True)


In window p000 we will translate the guest 0.0 Angstroms.
In window p001 we will translate the guest 1.8 Angstroms.
In window p002 we will translate the guest 3.6 Angstroms.
In window p003 we will translate the guest 5.3 Angstroms.
In window p004 we will translate the guest 7.1 Angstroms.
In window p005 we will translate the guest 8.9 Angstroms.
In window p006 we will translate the guest 10.7 Angstroms.
In window p007 we will translate the guest 12.4 Angstroms.
In window p008 we will translate the guest 14.2 Angstroms.
In window p009 we will translate the guest 16.0 Angstroms.


In [29]:
#    if not os.path.exists(f"tmp/windows/{window}/k-cl.pdb"):
#        structure.save(f"tmp/windows/{window}/k-cl.pdb")


## Optionally, tweak some parameters, like changing the charge of K+ to 1.3 and Cl- to -1.3

In [30]:
# Adjust K/Cl charge from +/- 1.0 to +/- 1.3

for window in window_list:
    structure = pmd.load_file(
        os.path.join(windows_directory, window, "k-cl.prmtop"),
        os.path.join(windows_directory, window, "k-cl.rst7"),
        structure=True,
    )
    for atom in structure.atoms:
        if atom.name == "Cl-":
            atom.charge = -1.3
        elif atom.name == "K+":
            atom.charge = 1.3
    structure.save(os.path.join(windows_directory, window, "k-cl.prmtop"), overwrite=True)
    structure.save(os.path.join(windows_directory, window, "k-cl.rst7"), overwrite=True)

## Solvate the structure in each window to the same number of waters

In [31]:
# Solvate in each window...

for window in window_list:
    print(f"Solvating window {window}...")

    if os.path.exists(os.path.join(windows_directory, window, "k-cl-sol.prmtop")):
        print("Skipping...")
        continue


    structure = pmd.load_file(
        os.path.join(windows_directory, window, "k-cl.prmtop"),
        os.path.join(windows_directory, window, "k-cl.rst7"),
        structure=True,
    )
    
    if not os.path.exists(os.path.join(windows_directory, window, "k-cl.pdb")):
        structure.save(os.path.join(windows_directory, window, "k-cl.pdb"))

    system = tleap.System()
    system.output_path = os.path.join(windows_directory, window)
    system.output_prefix = "k-cl-sol"

    system.target_waters = 2000
    system.neutralize = False
    system.template_lines = ["source leaprc.water.tip3p", "model = loadpdb k-cl.pdb"]
    system.build()

Solvating window a000...
Skipping...
Solvating window a001...
Skipping...
Solvating window a002...
Skipping...
Solvating window a003...
Skipping...
Solvating window p000...
Skipping...
Solvating window p001...
Skipping...
Solvating window p002...
Skipping...
Solvating window p003...
Skipping...
Solvating window p004...
Skipping...
Solvating window p005...
Skipping...
Solvating window p006...
Skipping...
Solvating window p007...
Skipping...
Solvating window p008...
Skipping...
Solvating window p009...
Skipping...


# Run the calculation

## Minimize, thermalize, equilibrate, and run production simulations

We have a few helper functions -- like `_config_pbc_min()` and `_config_pbc_md()` -- that help setup some smart defaults for AMBER. (I'll make a note to work on adding this for the OpenMM side of things.) The simulations can either be run directly, as indicated below, with `simulation.run()` or the input file can be written using `_amber_write_input_file()` and then wrapped using a cluster script (like PBS or whatever).

In [36]:
# Minimize

# for window in window_list:
#     simulation = amber.Simulation()
#     simulation.executable = "sander"

#     simulation.path = os.path.join(windows_directory, window)
#     simulation.prefix = "minimize"

#     simulation.inpcrd = "k-cl-sol.rst7"
#     simulation.ref = "k-cl-sol.rst7"
#     simulation.topology = "k-cl-sol.prmtop"
#     simulation.restraint_file = "disang.rest"

#     simulation.config_pbc_min()
#     simulation.cntrl["ntr"] = 1
#     simulation.cntrl["restraint_wt"] = 50.0
#     simulation.cntrl["restraintmask"] = "':1-2'"
#     print(f"Running minimization in window {window}...")
#     simulation.run()

# Simulate

for window in window_list:
    simulation = amber.Simulation()
    simulation.executable = "sander"

    simulation.path = os.path.join(windows_directory, window)
    simulation.prefix = "production"

    # simulation.inpcrd = "minimize.rst7"
    simulation.inpcrd = "k-cl-sol.rst7"
    simulation.ref = "k-cl-sol.rst7"
    simulation.topology = "k-cl-sol.prmtop"
    simulation.restraint_file = "disang.rest"

    simulation.config_pbc_md()
    simulation.cntrl["nstlim"] = 100
    simulation.cntrl["ntwx"] = 1

    print(f"Running production in window {window}...")
    simulation.run(overwrite=True)
 

2019-03-21 22:33:32,087: Writing production.in
2019-03-21 22:33:32,092: Running MD at k-cl/windows/a000
2019-03-21 22:33:32,092: Running MD at k-cl/windows/a000
2019-03-21 22:33:32,092: Running MD at k-cl/windows/a000
2019-03-21 22:33:32,108: Exec line: sander -O -p k-cl-sol.prmtop -ref k-cl-sol.rst7 -c k-cl-sol.rst7 -i production.in -o production.out -r production.rst7 -x production.nc -inf production.mdinfo -e production.mden


Running production in window a000...


2019-03-21 22:33:37,964: k-cl/windows/a000/production.out has TIMINGS
2019-03-21 22:33:37,966: MD completed ...
2019-03-21 22:33:37,966: MD completed ...
2019-03-21 22:33:37,966: MD completed ...
2019-03-21 22:33:37,972: Writing production.in
2019-03-21 22:33:37,975: Running MD at k-cl/windows/a001
2019-03-21 22:33:37,975: Running MD at k-cl/windows/a001
2019-03-21 22:33:37,975: Running MD at k-cl/windows/a001
2019-03-21 22:33:37,980: Exec line: sander -O -p k-cl-sol.prmtop -ref k-cl-sol.rst7 -c k-cl-sol.rst7 -i production.in -o production.out -r production.rst7 -x production.nc -inf production.mdinfo -e production.mden


Running production in window a001...


2019-03-21 22:33:46,582: k-cl/windows/a001/production.out has TIMINGS
2019-03-21 22:33:46,583: MD completed ...
2019-03-21 22:33:46,583: MD completed ...
2019-03-21 22:33:46,583: MD completed ...
2019-03-21 22:33:46,587: Writing production.in
2019-03-21 22:33:46,593: Running MD at k-cl/windows/a002
2019-03-21 22:33:46,593: Running MD at k-cl/windows/a002
2019-03-21 22:33:46,593: Running MD at k-cl/windows/a002
2019-03-21 22:33:46,598: Exec line: sander -O -p k-cl-sol.prmtop -ref k-cl-sol.rst7 -c k-cl-sol.rst7 -i production.in -o production.out -r production.rst7 -x production.nc -inf production.mdinfo -e production.mden


...

2019-03-21 22:35:19,996: k-cl/windows/p007/production.out has TIMINGS
2019-03-21 22:35:19,998: MD completed ...
2019-03-21 22:35:19,998: MD completed ...
2019-03-21 22:35:19,998: MD completed ...
2019-03-21 22:35:20,003: Writing production.in
2019-03-21 22:35:20,007: Running MD at k-cl/windows/p008
2019-03-21 22:35:20,007: Running MD at k-cl/windows/p008
2019-03-21 22:35:20,007: Running MD at k-cl/windows/p008
2019-03-21 22:35:20,011: Exec line: sander -O -p k-cl-sol.prmtop -ref k-cl-sol.rst7 -c k-cl-sol.rst7 -i production.in -o production.out -r production.rst7 -x production.nc -inf production.mdinfo -e production.mden


Running production in window p008...


2019-03-21 22:35:25,051: k-cl/windows/p008/production.out has TIMINGS
2019-03-21 22:35:25,054: MD completed ...
2019-03-21 22:35:25,054: MD completed ...
2019-03-21 22:35:25,054: MD completed ...
2019-03-21 22:35:25,058: Writing production.in
2019-03-21 22:35:25,070: Running MD at k-cl/windows/p009
2019-03-21 22:35:25,070: Running MD at k-cl/windows/p009
2019-03-21 22:35:25,070: Running MD at k-cl/windows/p009
2019-03-21 22:35:25,074: Exec line: sander -O -p k-cl-sol.prmtop -ref k-cl-sol.rst7 -c k-cl-sol.rst7 -i production.in -o production.out -r production.rst7 -x production.nc -inf production.mdinfo -e production.mden


Running production in window p009...


2019-03-21 22:35:35,513: k-cl/windows/p009/production.out has TIMINGS
2019-03-21 22:35:35,514: MD completed ...
2019-03-21 22:35:35,514: MD completed ...
2019-03-21 22:35:35,514: MD completed ...


# Analyze the simulation

## Setup the analysis

The analysis needs to know about:

- The parameter file that was used for the molecules,
- The simulation path,
- The trajectories, and
- The method to do the analysis (e.g., TI for the free energy with blocking analysis for the SEM)

In [37]:
free_energy = analysis.fe_calc()
free_energy.prmtop = "k-cl-sol.prmtop"
free_energy.trajectory = "production*.nc"
free_energy.path = windows_directory
free_energy.restraint_list = [restraint]
free_energy.collect_data()
free_energy.methods = ["ti-block"]
free_energy.ti_matrix = "diagonal"
free_energy.bootcycles = 100

2019-03-21 22:35:37,095: Replacing k-cl/windows/a004 with k-cl/windows/p000 in ['k-cl/windows/a000', 'k-cl/windows/a001', 'k-cl/windows/a002', 'k-cl/windows/a003', 'k-cl/windows/a004'] for `continuous_apr`...
2019-03-21 22:35:37,097: Load trajectories from k-cl/windows/a000...
2019-03-21 22:35:37,175: Loaded 100 frames...
2019-03-21 22:35:37,212: Load trajectories from k-cl/windows/a001...
2019-03-21 22:35:37,263: Loaded 100 frames...
2019-03-21 22:35:37,282: Load trajectories from k-cl/windows/a002...
2019-03-21 22:35:37,327: Loaded 100 frames...
2019-03-21 22:35:37,366: Load trajectories from k-cl/windows/a003...
2019-03-21 22:35:37,420: Loaded 100 frames...
2019-03-21 22:35:37,439: Load trajectories from k-cl/windows/p000...
2019-03-21 22:35:37,490: Loaded 100 frames...
2019-03-21 22:35:37,519: Load trajectories from k-cl/windows/p000...
2019-03-21 22:35:37,563: Loaded 100 frames...
2019-03-21 22:35:37,596: Load trajectories from k-cl/windows/p001...
2019-03-21 22:35:37,638: Loaded 

## Run the analysis and save the results

In [38]:
free_energy.compute_free_energy()
free_energy.compute_ref_state_work([restraint, None, None, None, None, None])

2019-03-21 22:35:41,781: Running ti-block analysis on attach phase ...
2019-03-21 22:35:41,808: Running bootstrap calculations
2019-03-21 22:35:41,810: Working on fraction ... 1.0
2019-03-21 22:35:41,889: Running ti-block analysis on pull phase ...
2019-03-21 22:35:41,928: Running bootstrap calculations
2019-03-21 22:35:41,930: Working on fraction ... 1.0
2019-03-21 22:35:42,022: Skipping free energy calculation for release


In [39]:
binding_affinity = -1 * (
    free_energy.results["attach"]["ti-block"]["fe"]
    + free_energy.results["pull"]["ti-block"]["fe"]
    + free_energy.results["ref_state_work"]
)

sem = np.sqrt(
    free_energy.results["attach"]["ti-block"]["sem"] ** 2
    + free_energy.results["pull"]["ti-block"]["sem"] ** 2
)

print(
    f"The binding affinity for K+ (+1.3) and Cl- (-1.3) = {binding_affinity:0.2f} +/- {sem:0.2f} kcal/mol"
)

The binding affinity for K+ (+1.3) and Cl- (-1.3) = 29.55 +/- 44.90 kcal/mol
